## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,78.76,69,1,18.14,clear sky
1,1,Victoria,HK,22.2855,114.1577,65.25,88,100,8.01,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,64.42,68,40,18.41,scattered clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,87.76,84,100,4.00,overcast clouds
4,4,Nikolskoye,RU,59.7035,30.7861,34.72,96,100,12.12,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
print("There were no empty rows")

There were no empty rows


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cidreira,BR,78.76,clear sky,-30.1811,-50.2056,
3,Puerto Ayora,EC,87.76,overcast clouds,-0.7393,-90.3518,
9,Hilo,US,77.11,few clouds,19.7297,-155.0900,
18,Sabang,ID,82.47,few clouds,5.8933,95.3214,
20,Kapaa,US,76.19,overcast clouds,22.0752,-159.3190,
24,Progreso,MX,87.91,overcast clouds,21.2833,-89.6667,
27,Castro,BR,80.06,few clouds,-24.7911,-50.0119,
28,Santa Cruz,BO,84.25,broken clouds,-17.8000,-63.1667,
30,Kahului,US,76.84,few clouds,20.8947,-156.4700,
31,Tamandare,BR,84.33,scattered clouds,-8.7597,-35.1047,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cidreira,BR,78.76,clear sky,-30.1811,-50.2056,Hotel Castelo
3,Puerto Ayora,EC,87.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Hilo,US,77.11,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Sabang,ID,82.47,few clouds,5.8933,95.3214,Hotel Citra
20,Kapaa,US,76.19,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
662,Alyangula,AU,75.07,broken clouds,-13.8483,136.4192,Groote Eylandt Lodge
663,Diamantino,BR,80.64,overcast clouds,-14.4086,-56.4461,Hotel Delta
665,Gizo,SB,80.82,overcast clouds,-8.1030,156.8419,Imagination Island
676,Karangampel,ID,77.61,light rain,-6.4622,108.4519,ROOFTOP WAG ALL


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</d1>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))